In [33]:
from PIL import Image, ImageDraw
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import math
import random

In [34]:
def sin(a):
    return math.sin(a)
def cos(a):
    return math.cos(a)
def tan(a):
    return math.tan(a)
pi=math.pi
sqrt2=math.sqrt(2)

def deg2rad(t):
    return t/180*pi
def rad2deg(t):
    return t/pi*180

def rotatePoint(p,c,t):
    '''
    rotate a point p=(px,py) about a point c=(cx,cy) by t radians counterclockwise
    '''
    px=p[0]; py=p[1]; px-=c[0]; py-=c[1]
    rx=px*cos(t)-py*sin(t)+c[0]
    ry=px*sin(t)+py*cos(t)+c[1]
    return (rx,ry)

def midpoint(a,b):
    return (a[0]/2+b[0]/2,a[1]/2+b[1]/2)

### Octagon Stuff

In [35]:
def OctPts(c, r, t):
    cx=c[0]; cy=c[1];
    points=[(cx,cy+r),(cx-r*sqrt2/2,cy+r*sqrt2/2),(cx-r,cy),(cx-r*sqrt2/2,cy-r*sqrt2/2),(cx,cy-r),(cx+r*sqrt2/2,cy-r*sqrt2/2),(cx+r,cy),(cx+r*sqrt2/2,cy+r*sqrt2/2)]
    for i in range(8): points[i]=rotatePoint(points[i],c,t)
    return points

def DrawBoard(c,r1,r2,t,w,draw):
    o1=OctPts(c,r1,t); draw.polygon(o1,outline='black',fill='white',width=w)
    o2=OctPts(c,r2,t); draw.polygon(o2,outline='black',fill='white',width=w)
    for i in range(8): draw.line([o1[i],o2[i]],fill='black',width=w)

### Chevron Stuff

In [36]:
def ChvPts(c,s,t):
    cx=c[0]; cy=c[1];
    points=[(cx,cy+s),(cx+s/2,cy+s/2),(cx+s/2,cy-s/2),(cx,cy),(cx-s/2,cy-s/2),(cx-s/2,cy+s/2)]
    for i in range(6): points[i]=rotatePoint(points[i],c,t)
    return points

def DrawChevron(c,s,t,draw,col):
    draw.polygon(ChvPts(c,s,t),fill=col)

### Generator

In [37]:
def GenN(n,draw,mdraw,rigged=75):
    '''image assumes 500x500 image, rescale as needed'''
    ocx=random.randrange(100,400); ocy=random.randrange(100,400) #octagon center x, octagon center y
    ocr=random.randrange(100,300); orr=random.uniform(1.3,3) #octagon radius, octagon radius ratio
    ofa=random.uniform(0,2*pi); oow=random.randrange(2,8); #octagon angle, outline width
    csz=random.randrange(25,40); #chevron size

    DrawBoard((ocx,ocy),ocr,ocr//orr,ofa,oow,draw)
    inOcto=Polygon(OctPts((ocx,ocy),ocr//orr,ofa))

    cspts=[]; cspolys=[]

    for i in range(n):
        ccx=random.randrange(ocx-rigged,ocx+rigged); ccy=random.randrange(ocy-rigged,ocy+rigged); #chevron center x,y
        cfa=random.uniform(0,2*pi); #chevron angle
        DrawChevron((ccx,ccy),csz,cfa,draw,"red"); DrawChevron((ccx,ccy),csz,cfa,mdraw,"white")
        cspts.append(ChvPts((ccx,ccy),csz,cfa)); cspolys.append(Polygon(ChvPts((ccx,ccy),csz,cfa))); 

    allDistinct=True;
    for i in range(n-1):
        for j in range(i,n):
            for cpt in cspts[i]:
                if cspolys[j].contains(Point(cpt[0],cpt[1])):allDistinct=False; break;
    return allDistinct

def Gen4(i):
    ad=False
    im=Image.open('bg/bluegradient500x500.jpeg');
    mim=Image.open('bg/pureblack500x500.png');
    while (ad==False):
        im=Image.open('bg/bluegradient500x500.jpeg'); draw=ImageDraw.Draw(im)
        mim=Image.open('bg/pureblack500x500.png'); mdraw=ImageDraw.Draw(mim)
        ad=GenN(4,draw,mdraw) 
    id='0'*(5-len(str(i)))+str(i)
    im.save('IMG'+id+'.jpeg','JPEG')
    mim.save('MSK'+id+'.jpeg','JPEG')

In [38]:
Gen4(10)